<a href="https://colab.research.google.com/github/sandeepthetechie/mastering_the_art_of_tensorflow/blob/master/Transfer_learning_with_TF_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning. 

In [ ]:
# Downloading the data. 
import zipref

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipref.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f'There are {len(dirnames)} directories and {len(filenames)} files in {dirpath}')

## Creating Data loaders

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"


train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("Training images: ")
train_data_10_percent = train_datagen.flow_from_directory(train_dir, 
                                                          target_size = IMAGE_SHAPE, 
                                                          batch_size = BATCH_SIZE, 
                                                          class_mode = "categorical")

print("Testing images: ")
test_data = test_datagen.flow_from_directory(test_data, 
                                             target_size = IMAGE_SHAPE, 
                                             batch_size = 32, 
                                             class_mode ="categorical")


# Setting up tensorboard call back. 

In [ ]:
import datetime

def create_tensorboard_callback(dir_name , experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f'Saving Tensorflow logs to {log_dir}.')
  return tensorboard_callback

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/1"
efficientnet_url = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
def create_model(model_url, num_classes = 10):
  feature_extractor_layer = hub.KerasLayer(model_url, 
                                           trainable = False, 
                                           name = "feature_extraction_layer",
                                           input_shape = IMAGE_SHAPE + (3,))
  model = tf.keras.Sequential([
    feature_extractor_layer, 
    layers.Dense(num_classes, activation = "softmax", name = "output_layer")
  ])

  return model
  

In [ ]:
resnet_model = create_model(resnet_url, num_classes = train_data_10_percent.num_classes)

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(loss = "categorical_crossentropy",
                     optimizer = tf.keras.optimizers.Adam(), 
                     metrics = ["accuracy"])


In [ ]:
resnet_history = resnet_model.fit(train_data_10_percent, 
                                  epochs = 5, 
                                  steps_per_epoch = len(train_data_10_percent),
                                  validation_data = test_data, 
                                  validation_steps = len(test_data),
                                  callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",
                                               experiment_name = "resnet50V2")])

# Plotting curves. 

In [ ]:
from matplotlib import pyplot as plt

def plot_loss_curve(history):
  """
  Returns separate loss curve for training and validation metrics 

  Args: 
    history: Tensorflow History object
  
  Returns: 
    Plots of training/validation loss and accuracy metrics
  """
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # Plot
  plt.plot(epochs, loss, label = "training_loss")
  plt.plot(epochs, val_loss, label = "val_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend

  # Plot accuracy

  plt.figure()
  plt.plot(epochs, accuracy, label = "training_accuracy")
  plt.plot(epochs, val_accuracy, label = "val_accuracy")
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend();

In [ ]:
plot_loss_curve(resnet_history)

# Creating and testing efficientnet. 

In [ ]:
efficientnet_model = create_model(model_url = efficientnet_url, 
                                  num_classes = train_10_percent_data)

efficientnet_model.compile(loss = "categorical_crossentropy",
                           optimizer = tf.keras.optimizers.Adam(), 
                           metrics = ["accuracy"])

efficientnet_history = efficientnet_model.fit(train_10_percent_data, 
                                              steps_per_epoch = len(train_10_percent_data),
                                              validation_data = test_data, 
                                              validation_steps = len(test_data), 
                                              callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",
                                                                                       experiment_name = "efficientnetb0")])

In [ ]:
plot_loss_curve(efficientnet_history)